# Dataset bajo Bag of Visual Words

Este notebook de Python tratará la creación del archivo con la cual se entrenará la red FeedForward para cada característica (si es vehículo o no, el tipo de vehículo y la marca del mismo), recuerde cambiar los datos con los cuales se hará al momento de inicializar el algoritmo.

In [22]:
#Importamos las librerías de costumbre
import numpy as np
import pandas as pd
#Instalamos la librería que nos va a permitir bajar de drive
!pip install gdown
import gdown
#Colocamos la URL del archivo
url = 'https://drive.google.com/uc?id=1tY5Nkrr7mEvkXjrlWPHyzx9DoDZhSD2l'
#Colocamos el nombre al archivo donde la información será guardada
Nombre = 'Vehiculos.rar'
#Lo descargamos
gdown.download(url,Nombre,quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1tY5Nkrr7mEvkXjrlWPHyzx9DoDZhSD2l
To: /home/slbejaranom/work/home/ConteoEIdentificacionDeTrafico/Feedforward/Vehiculos.rar
44.3MB [00:06, 7.08MB/s]


'Vehiculos.rar'

In [23]:
#Una vez descargado lo extraemos, para eso hacemos uso de las librerías patool y pyunpack
!pip install patool pyunpack
from pyunpack import Archive

#Extraemos
Archive('Vehiculos.rar').extractall('.')
print("Extraído satisfactoriamente")

Extraído satisfactoriamente


In [24]:
#Ahora vamos a pasar a la parte sucia, para eso primero instalamos opencv contrib
!pip install opencv-contrib-python
import cv2
import os

In [25]:
#Damos el nombre de la carpeta en la que se encuentra el dataset
dataset = "Vehiculos"
#Para las clases lo que hacemos es listar los nombres de las carpetas que se encuentran por dentro, se listan en orden alfabético
training_names = os.listdir(dataset)

#Ahora vamos a tener las variables para todas las imágenes
#La variable image_paths guardará las rutas de las imágenes
image_paths = []
image_classes = []
class_id = 0

def imglist(path):
    return [os.path.join(path,f) for f in os.listdir(path)]

for training_name in training_names:
    dir = os.path.join(dataset, training_name)
    class_path = imglist(dir)
    image_paths += class_path
    image_classes += [class_id]*len(class_path)
    class_id += 1

In [40]:
#Ahora crearemos una lista para tener la lista de descriptores que hallaremos a partir del SIFT
des_list = []
#Creamos el extractor SIFT con 50 puntos clave por imagen
sift = cv2.SIFT_create(50)
#Cogemos cada imagen, le pasamos el SIFT y los apilamos en des_list de la siguiente manera
#NOMBRE DEL ARCHIVO, DESCRIPTORES
for image_path in image_paths:
    im = cv2.imread(image_path)
    kpts, des = sift.detectAndCompute(im, None)
    des_list.append((image_path,des))

In [49]:
#Ahora vamos a apilar todos los descriptores de manera vertical
descriptors = des_list[0][1]
#Hay que descartar los descriptores que son None
for image_path, descriptor in des_list[1:]:
    if descriptor is not None:
        descriptors = np.vstack((descriptors, descriptor))
        
#Para hacer K Means es necesario que los datos estén en tipo float por lo que la siguiente linea es necesaria
descriptors_float = descriptors.astype(float)

## K Means
Acá viene la parte del K Means, en esta sección es necesario el estudio del codo en la gráfica para saber qué valor de K podríamos utilizar que no sea de gasto computacional pero que tampoco quite exactitud

In [ ]:
from scipy.cluster.vq import kmeans, vq

varianzas = []
num_clusters = range(50,2000,50)

for k in num_clusters:
    centroide, varianza = kmeans(descriptors_float, k, 1)
    varianzas.append(varianza)
    
elbow_plot_data = pd.DataFrame({'Numero de clusters':num_clusters,
                               'Varianza':varianzas})

import seaborn as sns
from matplotlib import pyplot as plt

sns.lineplot(x = 'Numero de Clusters',y = 'Varianza', data = elbow_plot_data)

plt.show()